In [1]:
from copy import deepcopy
from str_to_query import QueryBuilder

In [2]:
query_builder = QueryBuilder('dynamodb')

In [3]:
test_string_1 = '(a:1_)AND(b:2)AND(c:3)AND(d:4)'
test_string_2 = '((a:1_)AND(b:2))AND((c:3)AND(d:4))'

query_1 = query_builder(test_string_1)
query_2 = query_builder(test_string_2)

In [4]:
query_1._print_tree('inorder')
print('\n------\n')
query_2._print_tree('inorder')

<a : eq : 1_>
<AND>
<b : eq : 2>
<AND>
<c : eq : 3>
<AND>
<d : eq : 4>

------

<a : eq : 1_>
<AND>
<b : eq : 2>
<AND>
<c : eq : 3>
<AND>
<d : eq : 4>


In [5]:
query_1._print_tree('preorder')
print('\n------\n')
query_2._print_tree('preorder')

<AND>
<a : eq : 1_>
<AND>
<b : eq : 2>
<AND>
<c : eq : 3>
<d : eq : 4>

------

<AND>
<AND>
<a : eq : 1_>
<b : eq : 2>
<AND>
<c : eq : 3>
<d : eq : 4>


In [6]:
query_1._print_tree('postorder')
print('\n------\n')
query_2._print_tree('postorder')

<a : eq : 1_>
<b : eq : 2>
<c : eq : 3>
<d : eq : 4>
<AND>
<AND>
<AND>

------

<a : eq : 1_>
<b : eq : 2>
<AND>
<c : eq : 3>
<d : eq : 4>
<AND>
<AND>


<function _operator.and_>